In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

**STARTING SPARK SESSION**

In [0]:
spark = SparkSession.builder\
         .appName('store_analysis')\
             .getOrCreate()

**READING DATASETS/LOAD**

In [0]:
df_people = spark.read.csv('/FileStore/tables/store_data/superstore_people.csv', header=True, inferSchema=True)

df_orders = spark.read.csv('/FileStore/tables/store_data/superstore_orders.csv', header=True, inferSchema=True)

df_returns = spark.read.csv('/FileStore/tables/store_data/superstore_returns.csv', header=True, inferSchema=True)


In [0]:
df_people.show(5)
df_returns.show(5)

+-----------------+-------+
| Regional Manager| Region|
+-----------------+-------+
|  Sadie Pawthorne|   West|
|      Chuck Magee|   East|
|Roxanne Rodriguez|Central|
|      Fred Suzuki|  South|
+-----------------+-------+

+--------+--------------+
|Returned|      Order ID|
+--------+--------------+
|     Yes|US-2021-100762|
|     Yes|US-2021-100762|
|     Yes|US-2021-100762|
|     Yes|US-2021-100762|
|     Yes|US-2021-100867|
+--------+--------------+
only showing top 5 rows



In [0]:
df_orders.show(5)

+------+--------------+----------+----------+--------------+-----------+-------------+-----------+--------------+------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------------------+--------+--------+--------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|Customer Name|    Segment|Country/Region|        City|State/Province|Postal Code| Region|     Product ID|       Category|Sub-Category|        Product Name|               Sales|Quantity|Discount|  Profit|
+------+--------------+----------+----------+--------------+-----------+-------------+-----------+--------------+------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------------------+--------+--------+--------+
|     1|US-2021-103800|2021-01-03|2021-01-07|Standard Class|   DP-13000|Darren Powers|   Consumer| United States|     Houston|         Texas|      77095|Central|OFF-PA-1000

In [0]:
df_orders.printSchema()
df_orders.count()
print(len(df_orders.columns))

root
 |-- Row ID: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: date (nullable = true)
 |-- Ship Date: date (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State/Province: string (nullable = true)
 |-- Postal Code: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Sales: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Discount: string (nullable = true)
 |-- Profit: double (nullable = true)

21


In [0]:
df_orders.printSchema()

root
 |-- Row ID: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: date (nullable = true)
 |-- Ship Date: date (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State/Province: string (nullable = true)
 |-- Postal Code: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Sales: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Discount: string (nullable = true)
 |-- Profit: double (nullable = true)



**SALES & PROFIT **ANALYSIS****

In [0]:
sales_profit_df = df_orders.groupBy()\
                       .agg(
                           sum('Sales').cast('int').alias('total_sales_$'),
                           sum('Profit').cast('int').alias('total_profit_$'),
                           avg('Sales').cast('int').alias('avg_sales_$'),
                           avg('Profit').cast('int').alias('avg_profit_$')
                           )
                       

sales_profit_by_region = df_orders.groupBy('Region')\
                       .agg(
                           sum('Sales').cast('int').alias('total_sales_$'),
                           sum('Profit').cast('int').alias('total_profit_$'),
                           avg('Sales').cast('int').alias('avg_sales_$'),
                           avg('Profit').cast('int').alias('avg_profit_$')
                           )        


sales_profit_df.display()
sales_profit_by_region.display()

total_sales_$,total_profit_$,avg_sales_$,avg_profit_$
2301618,291611,232,28


Region,total_sales_$,total_profit_$,avg_sales_$,avg_profit_$
South,388983,46650,246,28
Central,499594,40323,220,17
East,685240,94963,236,31
West,727799,109674,231,33


In [0]:


most_sold_cat = df_orders.groupBy('Category')\
                         .agg(
                             sum('Sales').cast('int').alias('total_sales'),
                             sum('Profit').cast('int').alias('total_profit')
                             )

most_sold_cat.show()


+---------------+-----------+------------+
|       Category|total_sales|total_profit|
+---------------+-----------+------------+
|Office Supplies|     716212|      124179|
|      Furniture|     745794|       20965|
|     Technology|     839611|      146467|
+---------------+-----------+------------+



In [0]:
most_sold_cat_region = df_orders.groupBy('Region')\
                         .agg(
                             sum('Sales').cast('int').alias('total_sales'),
                             sum('Profit').cast('int').alias('total_profit'),
                             )\
                         .orderBy('total_sales', ascending = False)
                       
                         
most_sold_cat_region.show()                        


+-------+-----------+------------+
| Region|total_sales|total_profit|
+-------+-----------+------------+
|   West|     727799|      109674|
|   East|     685240|       94963|
|Central|     499594|       40323|
|  South|     388983|       46650|
+-------+-----------+------------+



In [0]:
order_region = df_orders.groupBy('Region')\
                        .agg(
                            count('Order ID').alias('total_orders')
                             ).orderBy('total_orders', ascending=False)
                        
order_region.display()
                      

Region,total_orders
West,3253
East,2986
Central,2335
South,1620


In [0]:
salesprofit_by_cat = df_orders.groupBy('Category','sub-Category')\
                               .agg(
                                   sum('Sales').cast('int').alias('total_sales'),
                                   sum('Profit').cast('int').alias('total_profit')
                               ).orderBy("total_sales",ascending=False)

salesprofit_by_cat.show(5, False)

+---------------+------------+-----------+------------+
|Category       |sub-Category|total_sales|total_profit|
+---------------+------------+-----------+------------+
|Furniture      |Chairs      |335768     |27223       |
|Technology     |Phones      |331560     |44974       |
|Office Supplies|Storage     |217604     |21536       |
|Furniture      |Tables      |208020     |-17753      |
|Office Supplies|Binders     |203710     |31256       |
+---------------+------------+-----------+------------+
only showing top 5 rows



**ORDER AND RETURN ANALYSIS**


In [0]:
df_orders_return = df_orders.join(df_returns, on='Order ID', how='left_outer')

print(len(df_orders_return.columns))

df_orders_return.printSchema()


22
root
 |-- Order ID: string (nullable = true)
 |-- Row ID: integer (nullable = true)
 |-- Order Date: date (nullable = true)
 |-- Ship Date: date (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State/Province: string (nullable = true)
 |-- Postal Code: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Sales: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Discount: string (nullable = true)
 |-- Profit: double (nullable = true)
 |-- Returned: string (nullable = true)



Analyze the impact of returns on profitability and identify products with the highest return rates.

In [0]:


return_over_profit = df_orders_return.groupBy('Product Name')\
                                     .agg(
                                         sum('Sales').cast('int').alias('total_sales'),
                                         count('Returned').alias('returns')
                                     ).orderBy('returns', ascending=False)




In [0]:
return_over_profit_sum = return_over_profit\
    .agg(
   sum(col('returns')).alias('total_returns')
        )
return_over_profit_sum.display()

total_returns
3226


Identify the best and worst-performing customer segments.


In [0]:
cust_segment_performance = df_orders.groupBy('Segment')\
                                    .agg(
                                        sum('Profit').cast('int').alias('total_profit'),
                                        sum('Sales').cast('int').alias('total_sales'),
                                        avg('Profit').cast('int').alias('avg_sales'),
                                    ).orderBy('total_profit', ascending = False).show()

+-----------+------------+-----------+---------+
|    Segment|total_profit|total_sales|avg_sales|
+-----------+------------+-----------+---------+
|   Consumer|      136709|    1159287|       25|
|  Corporate|       93690|     706236|       30|
|Home Office|       61210|     436094|       33|
+-----------+------------+-----------+---------+



Compare sales, profit, and returns across shipping modes.

In [0]:
shipping_mode_comparison = df_orders_return.groupBy('Ship Mode') \
    .agg(
        sum('Sales').cast('int').alias('total_sales'),
        sum('Profit').cast('int').alias('total_profit'),
        sum(when(col('Returned') == 'Yes', 1).otherwise(0)).alias('total_returns')
         ).orderBy('total_sales', ascending=False)
    

shipping_mode_comparison.show()

+--------------+-----------+------------+-------------+
|     Ship Mode|total_sales|total_profit|total_returns|
+--------------+-----------+------------+-------------+
|Standard Class|    1714115|      192830|         1794|
|  Second Class|     534999|       76419|          452|
|   First Class|     446620|       64254|          594|
|      Same Day|     203028|       33095|          386|
+--------------+-----------+------------+-------------+



Analyze sales and profit trends over time.

In [0]:
df_order_withdate = df_orders.withColumn('month_order', date_format('Order Date', 'yyyy-MM'))

sales_profit_overtime =  df_order_withdate.groupBy('month_order')\
                                           .agg(
                                             sum('Sales').cast('int').alias('total_sales'),
                                             sum('Profit').cast('int').alias('total_profit')
                                           ).orderBy('month_order', ascending =False).display()

month_order,total_sales,total_profit
2024-12,82573,8724
2024-11,117390,9507
2024-10,83204,10657
2024-09,86685,10831
2024-08,63652,9491
2024-07,45215,7335
2024-06,52316,8338
2024-05,44989,6405
2024-04,36277,953
2024-03,60595,14984


Identify highly profitable products and underperformers.

In [0]:
profit_by_product = df_orders.groupBy('Product Name')\
                             .agg(
                               sum('Profit').cast('int').alias('total_Profit')
                                   ).orderBy('total_Profit', ascending=False).show(3)
                             

least_perform = df_orders.groupBy('Product Name')\
                             .agg(
                               sum('Profit').cast('int').alias('total_Profit')
                                   ).orderBy('total_Profit', ascending=True).show(3)

+--------------------+------------+
|        Product Name|total_Profit|
+--------------------+------------+
|Canon imageCLASS ...|       25199|
|Fellowes PB500 El...|        7753|
|Hewlett Packard L...|        6983|
+--------------------+------------+
only showing top 3 rows

+--------------------+------------+
|        Product Name|total_Profit|
+--------------------+------------+
|Cubify CubeX 3D P...|       -8879|
|Lexmark MX611dhe ...|       -4589|
|Cubify CubeX 3D P...|       -3839|
+--------------------+------------+
only showing top 3 rows



**Determine the most loyal customers by analyzing repeat orders.**

In [0]:
loyal_cus = df_orders.groupBy('Customer id','Customer Name')\
                       .agg(
                         count('Order id').cast('int').alias('total_order')
                       ).orderBy('total_order', ascending=False).show()

+-----------+-------------------+-----------+
|Customer id|      Customer Name|total_order|
+-----------+-------------------+-----------+
|   WB-21850|      William Brown|         41|
|   BF-11170|         Ben Ferrer|         37|
|   GG-14650|       Greg Guthrie|         36|
|   PP-18955|         Paul Prost|         34|
|   JL-15835|           John Lee|         34|
|   XP-21865|       Xylona Preis|         34|
|   JW-15220|          Jane Waco|         34|
|   MA-17560|       Matt Abelman|         34|
|   JD-15895|   Jonathan Doherty|         32|
|   CK-12205|Chloris Kastensmidt|         32|
|   EH-13765|       Edward Hooks|         32|
|   SV-20365|        Seth Vernon|         32|
|   ZC-21910|   Zuschuss Carroll|         31|
|   AP-10915|     Arthur Prichep|         31|
|   EP-13915|         Emily Phan|         31|
|   JE-15715|         Joe Elijah|         30|
|   CT-11995|       Carol Triggs|         30|
|   LC-16870|      Lena Cacioppo|         30|
|   KL-16645|       Ken Lonsdale| 

**Identify regions with the highest return rates.**

In [0]:
high_return_region = df_orders_return.groupBy('Region')\
                                     .agg(
                                       count('Order ID').alias('total_orders'),
                                       count('Returned').alias('total_return'),
                                          )\
                                       .withColumn('return_rate',(col('total_return') / col('total_orders') * 100 ).cast('int'))\
                                          .orderBy('total_return', ascending=False).display()

Region,total_orders,total_return,return_rate
West,4667,1904,40
East,3594,757,21
Central,2529,286,11
South,1830,279,15


In [0]:
%fs rm FileStore/tables/sample_text_with_header.csv



res0: Boolean = false

In [0]:
%fs ls /FileStore/tables/parquetData/df_ppl.parqute/

path,name,size,modificationTime
dbfs:/FileStore/tables/parquetData/df_ppl.parqute/_SUCCESS,_SUCCESS,0,1736850088000
dbfs:/FileStore/tables/parquetData/df_ppl.parqute/_committed_8185865590075153001,_committed_8185865590075153001,123,1736834953000
dbfs:/FileStore/tables/parquetData/df_ppl.parqute/_committed_8562446173328730499,_committed_8562446173328730499,232,1736850087000
dbfs:/FileStore/tables/parquetData/df_ppl.parqute/_committed_vacuum6325623301711621334,_committed_vacuum6325623301711621334,96,1736850088000
dbfs:/FileStore/tables/parquetData/df_ppl.parqute/_started_8562446173328730499,_started_8562446173328730499,0,1736850084000
dbfs:/FileStore/tables/parquetData/df_ppl.parqute/part-00000-tid-8562446173328730499-8282347d-7ea2-4539-8ccb-9fc96237a0c6-47-1-c000.snappy.parquet,part-00000-tid-8562446173328730499-8282347d-7ea2-4539-8ccb-9fc96237a0c6-47-1-c000.snappy.parquet,985,1736850087000


In [0]:
df_people.write.parquet('FileStore/tables/parquetData/df_ppl.parqute', mode='overwrite')